Помимо атрибутов объектов, в классе могут быть определены также и атрибуты класса: поля класса, методы класса. Кроме того, в классе можно определить и **статические методы** (*static methods*) – функции не связанные с данными объектов или класса.

## Методы класса

Для того, чтобы определить метод класса, необходимо применить декоратор `@classmethod`. Благодаря этому декоратору, в соответствии с протоколом передачи аргументов для методов класса, в метод первым аргументом будет передаваться класс, а не объект (как в случае с методом объекта). Поэтому, аналогично тому, как при вызове метода объекта первому параметру (`self`) присваивается сам объект, у которого вызван метод, при вызове метода класса, первому параметру автоматически передается сам класс. Поэтому, первый параметр метода класса принято называть `cls`. Через этот объект можно получать доступ к полям класса. 

>Когда мы говорим объект класса `A`, обычно подразумевается объект тип которого – класс `A`, т.е. экземпляр класса `A`. Но класс `A` сам по себе является объектом типа `type`. В данном случае параметр `cls` получает класс как объект, а не экземпляр класса `A`.

Метод класса может быть вызван как через имя класса, так и через имя объекта (или через `self`).

In [ ]:
class Rectangle:
    MAX_VALUE = 100             # поле класса

    def __init__(self, width, height) -> None:
        # вызов метода класса в методе объекта:
        #  self.validate()  <=>   Rectangle.validate()
        if self.validate(width) and Rectangle.validate(height):
            self.width = width    #  поле объекта
            self.height = height  #  поле объекта
        else:
            self.width = self.height = 0

    @classmethod
    def setValue(cls, max_value):
        if max_value < 0:
            raise ValueError("Значение должно быть не меньше 0")
        cls.MAX_VALUE = max_value

    @classmethod
    def validate(cls, value):
        return 0 <= value <= cls.MAX_VALUE

In [ ]:
Rectangle.MAX_VALUE     # поле класса

In [ ]:
r = Rectangle(20, 30)

In [ ]:
r.width = 4

In [ ]:
r = Rectangle(20, 30)
r.MAX_VALUE

In [ ]:
Rectangle.MAX_VALUE = "43"
Rectangle.MAX_VALUE

При этом можно считать, что класс в результате такого изменения оказывается "сломанным": при попытке создать новый объект, мы получим ошибку, так как метод `validate()` будет теперь пытаться произвести сравнение числа и строки.

In [ ]:
# r2 = Rectangle(2, 5)

Метод класса `setValue()` позволяет безопасно установить значение `MAX_VALUE`, без риска сломать класса.

In [ ]:
Rectangle.setValue("43") # вызов метода класса через класс
Rectangle.MAX_VALUE


In [ ]:
r.setValue(400)     # вызов метода класса через объект
Rectangle.MAX_VALUE

## Статические методы

Для определения статического метода используется декоратор ``@staticmethod``. В отличие от методов объекта и методов класса, статическому методу не передаётся первым аргументом ни экземпляр (`self`), ни класс (`cls`).

Статический метод – это, по сути, обычная функция, логически связанная с классом, но не имеющая доступа к его состоянию. Такие методы используются для вспомогательных операций.

В примере ниже метод `get_area()` вычисляет площадь по переданным аргументам, не обращаясь ни к каким атрибутам класса или объекта:

In [ ]:
class Rectangle:
    MAX_VALUE = 100             # публичное поле класса

    def __init__(self, width, height) -> None:
        self.width = width
        self.height = height
        # вызов статического метода внутри метода объекта:
        self.area = self.get_area(width, height)

    @staticmethod
    def get_area(width, height):
        return width * height

In [ ]:
r = Rectangle(3, 4)

Rectangle.get_area(3, 4)    # вызов статического метода через класс
r.get_area(3, 4)            # вызов статического метода через объект

>Зачем нужен декоратор? Без `@staticmethod` функция, определённая в теле класса, будет работать корректно только при вызове через имя класса. При вызове через экземпляр (`r.get_area(...)`) или через `self` Python автоматически передаст объект первым аргументом — это стандартный протокол вызова методов. В результате функция получит лишний аргумент и завершится с ошибкой. Декоратор `@staticmethod` отключает это поведение, позволяя вызывать метод единообразно в любом контексте.

## Порядок поиска полей

Доступ к полям класса можно получить через экземпляр класса. Но что если одновременно имеются и поле класса и поле объекта с одним и тем же именем? Python ищет атрибут по определенной схеме (*attribute lookup order*), в соответствии с которой имя ищется сначала в пространстве имен объекта, затем в пространстве имен класса. При наличии дескрипторов, схема поиска имен несколько сложнее. Это описано в файле про дескрипторы.

In [ ]:
class Example:
    width = 100  # атрибут класса

Example.__dict__    # пространство имен класса

In [ ]:
obj = Example()
obj.width

В пространстве имен объекта имя `width` отсутствует, так как атрибута с таким именем у объекта нет: 

In [ ]:
obj.__dict__    # пространство имен объекта

Обращаясь к полю класса через экземпляр класса можно только получить значение, но нельзя установить значение. Если выполнить инструкцию `obj.width = 200`, то будет создано поле объекта с именем `width` и именно ему будет присовено значение `200`. Значение поля `width` класса `Example` при этом останется прежним:

In [ ]:
obj.width = 200
obj.__dict__

In [ ]:
Example.width, obj.width